In [ ]:
#Import des modules nécessaires à savoir requests pour créer,utiliser et excécuter nos requêtes, selenium comme recommandé, time pour permettre le chargement des pages du site et sys pour pouvoir quitter le systeme de la boucle dès qu'indiquer. 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import sys

In [ ]:
#chemin complet vers chromedriver.exe
path_to_chromedriver = '../ChromeDriver-Win32/chromedriver-win32/chromedriver.exe'

# Création d'une instance de Service en spécifiant le chemin du chromedriver
chrome_service = Service(path_to_chromedriver)

# Création du navigateur en utilisant l'instance de Service
driver = webdriver.Chrome(service=chrome_service)

# Naviguation vers l'URL souhaitée
driver.get('https://nekketsu.store/collections/bague-japonaise')


In [ ]:
#Recherche du nombre de page suivante en accédant à la classe "page"
page_suivante = driver.find_elements(By.XPATH, "//div[@class='paginate']//span[@class='page']")

nb_pages = len(page_suivante) #Nombre d'éléments stocker dans la variable nb_pages
print("Nombre de pages dans la catégorie : " + str(nb_pages + 1)) #Affichage du nombre de pages présentes pour la catégorie séléctionner

In [ ]:
produits = [] #Création du tableau où seront stocker les données.

In [ ]:
j = 2 #Initialisation d'une variable j qui sera incrémenter à chaque tour de la boucle afin de modifier les liens des pages et accéder aux produits.
for page in range(nb_pages+1): #Pour le nombre de pages présentes dans la catégorie
    i = 1 #Initialisation d'une variable i qui sera incrémenter à chaque tour de la boucle afin d'accéder aux diverses produits présents.
    print("Page " + str(j-1) + ":") #Affichage de la page en cours de scrapping
    time.sleep(3) #Attente de 3 sec permettant le bon chargement de la page
    for data in range(300):  #Limitation des produits à afficher
        try: #Essayer de...
            #Récupérer les diverses données
            categorie = driver.find_element("xpath", f'/html/body/div[4]/div/div[2]/div[1]/h1').text 
            produit = driver.find_element("xpath", f'/html/body/div[4]/div[3]/div[4]/div[1]/div/div[1]/div{[i]}/div/a/div/h2').text            
            prix_et_reduction = driver.find_element("xpath", f'/html/body/div[4]/div[3]/div[4]/div[1]/div/div[1]/div{[i]}/div/a/div/span').text 
            prix = prix_et_reduction.split(' ')[0] #Suppression du prix initiale afin de garder le prix actuel du produit
            image_element = driver.find_element("xpath", f'/html/body/div[4]/div[3]/div[4]/div[1]/div/div[1]/div{[i]}/div/div/a/div/div/img')
            image_src = image_element.get_attribute('src')
            
            #Afficher les diverses données
            print(categorie)
            print(produit)
            print(prix)
            print(image_src)
            
            #Stocker les diverses données dans la table
            produits.append([categorie, produit, prix, image_src])
        
        except Exception: #Si l'essaye échoue..
            pass #Ne plus rien faire
        #Incrémenter l'indice i de 1 sauf s'il a déja atteint une valeur de 20 (en effet sur ce site une page ne peut contenir que 20 produits)
        i += 1
        if i > 32:
            break  # Sort de la boucle interne si i dépasse 20
    
    
    try: #Essayer de...
        suivant = driver.get(f'https://nekketsu.store/collections/t-shirt-japonais?page={j}') #Passer à la page suivante
    except: #Si cela ne fonctionne pas
        print("Toutes les pages ont été scrappé") #Afficher ceci
        sys.exit() #Et sortir du système de la boucle pour réduire les itérations supplémentaires
    j += 1 #Incrémenter l'indice j pour passer à la page suivante au prochain tour


print("Actuellement, " + str(len(produits)) + " données ont pu être récupérées.")  #Afficher le nombre d'élément présente dans la liste

In [ ]:
# Utilisation de join pour séparer par des points-virgules
resultat = ";".join([";".join(map(str, element)) for element in produits]) #Séparer les données récupérées par des ";" afin de pouvoir récupérer les données, les stocker dans une table CSV et les importer dans une base de données
print(resultat) # Affichage du résultat

driver.quit() # Fermer le navigateur 